In [42]:
#テキストの属性をもとに段落を判別しつつテキストデータを取得

import fitz  # PyMuPDF
import re

# PDFファイルのパス
pdf_path = "ir/2025/34070_旭化成/24jp.pdf"

# テキストクリーニング関数
def clean_text_by_attributes(page):
    text = ""
    blocks = page.get_text("dict")["blocks"]
    previous_font = None
    previous_size = None
    previous_color = None
    previous_bbox = None
    #previous_width = None
    previous_text_type = None

    def get_threshold_by_font_size(font_size):
        # フォントサイズによって閾値を動的に変更
        #if font_size <= 10:
        #    return 10  # 小さいフォントサイズ
        #elif font_size <= 14:
        #    return 14  # 中程度のフォントサイズ
        #else:
        #    return 24  # 大きいフォントサイズ
        return font_size * 2
    
    for block in blocks:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                font = span["font"]
                size = span["size"]
                color = span["color"]
                bbox = span["bbox"]
                #width = span["width"]
                text_type = span.get("text_type", "text")  # デフォルトで"text"
                content = span["text"]

                # \nの直前にある半角スペースを除外
                content = re.sub(r" \n", "\n", content)
                content = re.sub(r"^ ", "", content)

                threshold = get_threshold_by_font_size(size)  # フォントサイズに基づく閾値
                
                # フォント、サイズ、色、位置、幅、タイプのいずれかが変化した場合は段落の区切りと見なす
                if (
                    #previous_font != font or
                    #previous_size != size or
                    #previous_color != color or
                    (previous_bbox and abs(bbox[3] - previous_bbox[3]) > threshold) or  # 縦位置の大きな変化
                    #previous_width != width or
                    previous_text_type != text_type #or
                    #content != "\n"
                    
                ):
                    text += "\n\n"  # 新しい段落の開始
                else:
                    text += "■"  # 同じ段落と見なしてスペースで連結

                text += content
                previous_font = font
                previous_size = size
                previous_color = color
                previous_bbox = bbox
                #previous_width = width
                previous_text_type = text_type

    # 最終的な改行の調整
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text

# PDFファイルのオープン
doc = fitz.open(pdf_path)

# PDFからテキストを抽出し、クリーニング
cleaned_text = ""

for page_number, page in enumerate(doc, start=0):
    # ページ開始タグ
    cleaned_text += f"<page {page_number}>\n"
    # ページ内容
    cleaned_text += clean_text_by_attributes(page) + "\n"
    # ページ終了タグ
    cleaned_text += f"</page {page_number}>\n\n"

# 出力ファイルパス
output_file_path = "ir/2025/34070_旭化成/cleaned_extracted_text_by_attributes.txt"

# ファイルへの書き込み
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"ファイルが保存されました: {output_file_path}")


ファイルが保存されました: ir/2025/34070_旭化成/cleaned_extracted_text_by_attributes.txt
